In [1]:
import pickle
import numpy as np

In [2]:
X = pickle.load(open('dbscan/data_10points_10dims.dat', 'rb'), encoding='latin1')

In [3]:
X.todense()

matrix([[0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
        [0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
        [0, 0, 0, 0, 1, 0, 1, 1, 1, 0],
        [1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
        [1, 1, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 1, 1, 0, 1, 1, 0, 1]])

In [4]:
X[0,:].todense(),X[2,:].todense()

(matrix([[0, 0, 0, 1, 1, 0, 1, 1, 0, 1]]),
 matrix([[0, 0, 0, 1, 0, 1, 0, 0, 0, 1]]))

In [5]:
p1 = X[0,:]
p2 = X[2,:]

In [6]:
def jaccard(p1,p2):
    diff = (p1!=p2).sum()
    return diff/(diff+p1.multiply(p2).sum())

In [7]:
from sklearn.metrics.pairwise import pairwise_distances
pairwise_distances(p1.todense(),p2.todense(),'jaccard'),jaccard(p1,p2)

(array([[ 0.66666667]]), 0.66666666666666663)

In [8]:
def region_query(p,eps,X):
    return np.array([i for (i,x) in enumerate(X) if jaccard(p,x) <= eps])

In [9]:
neighbors = region_query(p1,0.01,X)

In [10]:
mask = np.ones(X.shape[0],dtype=bool)
mask[neighbors] = False

In [11]:
X

<10x10 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [12]:
X[mask]

<6x10 sparse matrix of type '<class 'numpy.int64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [13]:
def dbscan(X, eps, MinPts, clustering = np.zeros(X.shape[0]), nclusters = 0):
    if all(clustering):
        return clustering
    else:
        current_point = X[clustering[clustering>0][0]] # first unassigned point
        neighbors = region_query(current_point,eps,X)
        if len(neighbors) >= MinPts: # if it's a core point
            nclusters += 1
            clustering[current_point] = nclusters
            clustering[neighbors] = nclusters
            for point in nieghbors:
                if not clustering[point]:
        else:
            clustering[current_point] = -1 # outlier

IndentationError: expected an indented block (<ipython-input-13-adcf69fec9b5>, line 13)

In [18]:
def density_reachable(p,X,eps,MinPts,clustering,nclusters):
    neighbors = region_query(p,eps,X)
    if len(neighbors) >= MinPts:
        clustering[neighbors] = nclusters
        return np.array([density_reachable(X[p],X,eps,MinPts,clustering,nclusters) for p in neighbors if not clustering[p]])
    else:
        return neighbors

In [19]:
density_reachable(X[2],X,0.4,2,np.zeros(X.shape[0]),nclusters=1)

array([], dtype=float64)

In [20]:
region_query(X[2],0.4,X)

array([2, 5])

In [21]:
region_query(X[5],0.4,X)

array([2, 5, 6])

In [22]:
region_query(X[6],0.4,X)

array([5, 6])

In [23]:
region_query(X[3],0.1,X)

array([0, 1, 3, 9])

In [24]:
mask = np.ones(X.shape[0],dtype=bool)
mask[region_query(X[2],0.4,X)] = False